In [1]:
import nltk
nltk.download('punkt')
from collections import Counter, defaultdict
import random
import heapq
import operator
import math
from nltk.corpus import arcosg

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brennanium/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def load_file(data_file=None):
    sentences = []   
    for sent in arcosg.sents() if data_file is None else arcosg.sents(data_file):
        sentences.append(["<s>"] + sent + ["</s>"])
    return sentences

In [3]:
print(arcosg.tagged_words())
print(arcosg.tagged_words(tagset='universal'))

[('[3]', 'XSC'), ('ach', 'CC'), ('bha', 'V-S'), ...]
[('[3]', 'X'), ('ach', 'CONJ'), ('bha', 'VERB'), ...]


In [4]:
# arcosg.sents('c01.txt')
# sentences = load_file(data_file='c01.txt')
sentences = load_file()
print(sentences)

s', 'Nicholl', "a'", 'tighinn', 'a-mach', 'mu', 'choinneamh', '</s>'], ['<s>', 'Spasitch', 'ga', 'gluasad', 'a-steach', 'gu', 'Yositch', '</s>'], ['<s>', 'Yositch', "a'", 'ruith', 'leatha', '</s>'], ['<s>', 'is', 'Johnson', 'còmhla', 'ris', '</s>'], ['<s>', 'chuir', 'esan', 'gu', 'Bostardivitch', 'i', '</s>'], ['<s>', 'ach', 'choinnich', 'MacStay', 'i', '</s>'], ['<s>', 'MacStay', 'gu', 'MacClare', '</s>'], ['<s>', 'MacClare', "a's", 'a’', 'chearcall', 'meadhan', '</s>'], ['<s>', 'chan', 'eil', 'duine', 'aige', 'ach', 'e', 'fhèin', '</s>'], ['<s>', 'tha', 'e', 'fhèin', "'", 'dol', 'a', 'dh’', 'fheuchainn', 'às', '</s>'], ['<s>', 'tha', 'Johnson', 'còmhla', 'ris', 'air', 'an', 'loighne', 'ann_an_seo', '</s>'], ['<s>', 'Johnson', "a'", 'gluasad', 'suas', '</s>'], ['<s>', 'Johnson', 'air', 'an', 'loighne', '</s>'], ['<s>', "'s", 'tha', 'e', 'tha', 'e', "a'", 'cumail', 'a-muigh', 'is', "a'", 'feitheamh', '</s>'], ['<s>', 'MacClare', 'còmhla', 'ris', '</s>'], ['<s>', 'tha', 'e', "'ga", 'feu

In [5]:
unigrams = [ug for sent in sentences for ug in sent]

unigram_fdist = nltk.FreqDist(unigrams)

unigram_count = len(unigrams)
for ug, freq in unigram_fdist.most_common(50): 
    print(f"{str(ug):<15} {freq/unigram_count}")

<s>             0.10013965710505522
</s>            0.10013965710505522
a               0.0372388320255793
an              0.02410003858946324
air             0.019983829177309394
,               0.018256491298994836
e               0.016850732281004796
.               0.016694536834561458
a'              0.014177033756592367
tha             0.014112717984527463
agus            0.013846266928830004
na              0.012532387585218399
gu              0.010391591172200886
a’              0.008857200610081039
bha             0.0087010051636377
's              0.008691817196199856
iad             0.007910839963983167
ach             0.005751667616089968
robh            0.005163637700067991
aig             0.005062570058251714
ann             0.004952314448997593
sin             0.0046031716863595435
i               0.004456164207354049
do              0.004281592826035024
"               0.004208089086532277
am              0.0041253973795916865
gun             0.004107021444716
mi       

In [6]:
bigrams = [bg for sent in sentences for bg in nltk.bigrams(sent)]

bigram_fdist = nltk.FreqDist(bigrams)

bg_count = len(bigrams)
for bg, freq in bigram_fdist.most_common(100): 
    probability = freq / unigram_fdist[bg[0]] # P(bg[1]|bg[0]) = count(bg) / count(bg[0])
    print(f"{str(bg):<30} {probability}")

('.', '</s>')                  0.9372592184920198
('<s>', 'a')                   0.11771722176346454
('<s>', 'agus')                0.08523717772272686
(',', '</s>')                  0.41821841972823354
('<s>', "'s")                  0.04459124690338563
('a', 'tha')                   0.10066617320503331
('<s>', '[1]')                 0.036700614735296816
('<s>', 'ach')                 0.03615010551426737
('a', 'bha')                   0.09696521095484827
('<s>', 'tha')                 0.03211303789338472
('<s>', '[2]')                 0.03156252867235526
('?', '</s>')                  0.8826666666666667
('<s>', 'Tha')                 0.029085237177722725
('air', 'an')                  0.13655172413793104
('<s>', 'gu')                  0.02559867877786953
('<s>', 'nach')                0.025139921093678318
('<s>', 'gun')                 0.02477291494632535
('na', 'h-')                   0.18475073313782991
('<s>', '[3]')                 0.02284613267272227
('an', 't-')                  

In [34]:
fivegrams = [fg for sent in sentences for fg in nltk.ngrams(['<s>' for _ in range(5-2)] + sent,5)]

print(fivegrams[0:20])

[('<s>', '<s>', '<s>', '<s>', '[3]'), ('<s>', '<s>', '<s>', '[3]', 'ach'), ('<s>', '<s>', '[3]', 'ach', 'bha'), ('<s>', '[3]', 'ach', 'bha', 'e'), ('[3]', 'ach', 'bha', 'e', 'neònach'), ('ach', 'bha', 'e', 'neònach', '</s>'), ('<s>', '<s>', '<s>', '<s>', 'nach'), ('<s>', '<s>', '<s>', 'nach', 'do'), ('<s>', '<s>', 'nach', 'do', 'test'), ('<s>', 'nach', 'do', 'test', 'iad'), ('nach', 'do', 'test', 'iad', 'na'), ('do', 'test', 'iad', 'na', '[?]'), ('test', 'iad', 'na', '[?]', 'leis'), ('iad', 'na', '[?]', 'leis', "a'"), ('na', '[?]', 'leis', "a'", 'vet'), ('[?]', 'leis', "a'", 'vet', '-'), ('leis', "a'", 'vet', '-', '</s>'), ('<s>', '<s>', '<s>', '<s>', 'cha'), ('<s>', '<s>', '<s>', 'cha', 'robh'), ('<s>', '<s>', 'cha', 'robh', 'duine')]


In [8]:
trigrams = [tg for sent in sentences for tg in nltk.trigrams(["<s>"] + sent)]

trigram_fdist = nltk.FreqDist(trigrams)

# condition_pairs = (((w0, w1), w2) for w0, w1, w2 in trigrams)
# trigram_probabilities = nltk.ConditionalFreqDist(condition_pairs)

tg_count = len(trigrams)
num_of_sents = len(sentences)
for tg, freq in trigram_fdist.most_common(50): 
    if tg[:2] == ("<s>","<s>"):
        probability = freq / num_of_sents
    else: 
        probability = freq / bigram_fdist[(tg[0],tg[1])] # P(tg[2]|tg[0],tg[1]) = count(tg) / count(tg[0],tg[1])
    
    print(f"{str(tg):<35} {probability}")

('<s>', '<s>', 'a')                 0.11771722176346454
('<s>', '<s>', 'agus')              0.08523717772272686
('<s>', '<s>', "'s")                0.04459124690338563
('<s>', '<s>', '[1]')               0.036700614735296816
('<s>', '<s>', 'ach')               0.03615010551426737
('<s>', '<s>', 'tha')               0.03211303789338472
('<s>', '<s>', '[2]')               0.03156252867235526
('<s>', '<s>', 'Tha')               0.029085237177722725
('<s>', 'a', 'tha')                 0.2190179267342167
('<s>', '<s>', 'gu')                0.02559867877786953
('<s>', '<s>', 'nach')              0.025139921093678318
('<s>', '<s>', 'gun')               0.02477291494632535
('<s>', '<s>', '[3]')               0.02284613267272227
('<s>', 'a', 'bha')                 0.16835541699142634
('<s>', '<s>', '“')                 0.017249288925589502
('<s>', '<s>', '[4]')               0.015414258188824663
('<s>', '<s>', 'bha')               0.015322506651986421
('<s>', 'gu', 'bheil')              0.56272

In [9]:
num_sentences = 10
max_sent_len = 30
random.seed()

def print_sentences(sent_list):
    for sent, prob in sent_list:
        for word in sent:
            print(word,end=" ")
        print("\nprobability of: ", prob)
        print("\n\n")

def train_ngram(ngrams):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for ng in ngrams:
        model[ng[:-1]][ng[-1]] += 1  # [(w1, w2, ... , wn-1)][wn]
    
    total_count = 0      
    for hist in model: # hist = (w1, w2, ... , wn-1)
        total_count = float(sum(model[hist].values()))
        for w in model[hist]: # w = wn
            model[hist][w] /= total_count
    
    return model

def ngram_random_sentence(model, seed, n=50):
    # how far are you looking back from current word; bigram -> 1, trigram -> 2, etc.
    hist_size = len(seed) * -1

    # max number of tries to get a value that is not <UNK>
    max_tries = 100

    # probability of the sentence
    probability = 1.0

    sentence = seed

    while len(sentence) <= max_sent_len:
        # hist_size number of words before current word
        given = tuple(w for w in sentence[hist_size:])

        # get n most probable next words from the model[given], P(word|given)
        most_probable = heapq.nlargest(n, model[given].items(), key=operator.itemgetter(1))

        # choose one (word, probability) pair
        choice = random.choice(most_probable)

        tries = 0
        while choice[0] == "<UNK>" and tries <= max_tries:
            choice = random.choice(most_probable)
            most_probable = heapq.nlargest(n * 2, model[given].items(), key=operator.itemgetter(1))
            tries += 1

        sentence.append(choice[0])
        probability *= choice[1]

        if sentence[-1] == "</s>":
            break


    if not sentence[-1] == "</s>":
        sentence.append("</s>")
    
    return sentence, probability

In [10]:
unigram_model = {}
for ug, freq in unigram_fdist.most_common(200): 
    unigram_model[ug] = freq/unigram_count

In [11]:
unigram_sentences = []

def unigram_random_sentence(unigram_model):
    probability = unigram_model["<s>"]

    sentence = ["<s>"]

    while len(sentence) <= max_sent_len:
        # choose one (word, probability) pair
        choice = random.choice(list(unigram_model.items()))

        while choice[0] == "<s>":
            choice = random.choice(list(unigram_model.items()))
        
        sentence.append(choice[0])
        probability *= choice[1]

        if sentence[-1] == "</s>":
            break

    if not sentence[-1] == "</s>":
        sentence.append("</s>")
        probability *= unigram_model["</s>"]

    return sentence, probability


#generate the sentences
for index in range(0,num_sentences):
    test, probability = unigram_random_sentence(unigram_model)

    unigram_sentences.append((test,probability))
    
#TODO: generate 10 sentences using the unigram model. Suppose maximum sentence length = 30
    
print_sentences(unigram_sentences)

<s> thàinig faighinn [3] gur buille cho . sìos obair againn buille bha mòr robh robh no sinn cur [2] a's ga ann 'n , bhiodh leis thàinig [2] ga fear </s> 
probability of:  1.379615613644665e-85



<s> sa le airson mi còmhla sam_bith sa aig cha chuir is rud feuchainn dol ag a-mach agus agus bhith nan fhathast ma sa air ùr leis gum ris bliadhna eadar </s> 
probability of:  5.594245993586694e-85



<s> ise Chaidh thuirt dhan mar idir nan 's mu Agus ' agus cò - Cha ma tu fhalbh mise thàinig suas fhalbh Cha seo ' seachad ars’ a nuair coimhead </s> 
probability of:  1.0434749732705015e-87



<s> 's is aige a_h-uile a's sin ’s dè sìos e às ò [Name] dè na anns ris math eadar fhéin sinn thall tha eil mise dè fear coimhead dòcha bho </s> 
probability of:  7.035318291737444e-86



<s> dh' sìos no dìreach de ; cò ma-thà bidh ais trì dòcha gun [Placename] mu duine robh ach Agus an-diugh [1] . bu ach an-diugh ais i o ’s dèanamh </s> 
probability of:  1.3208946396880586e-88



<s> [3] Agus iad gus ro

In [12]:
bigram_model = train_ngram(bigrams)

In [13]:
bigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(bigram_model, ["<s>"])

    bigram_sentences.append((test,probability))

print_sentences(bigram_sentences)

<s> Agus gu Bostardivitch gu Spasitch </s> 
probability of:  6.407477641332326e-11



<s> tha dol do dhuine a' tionndadh seachad na bha ann de dhà arbhar is b' àirde na gillean . 2 m ; ann_an Gàidhlig na cluicheadairean nas caime , Alasdair </s> 
probability of:  1.905872252313277e-52



<s> Bha seacaid aige , ri fichead agus bha air n- ais dha an taigh 'gan draibheadh a-nisd ga fheuchainn rudeigin òg 'ga leanail . 6 . " mar dhearbhadh ? </s> 
probability of:  2.367303544219488e-46



<s> bha a’ bhàrdachd Mairearaid . " feuch dé thuirt mi mo chadal . " , Sir lain , bu luaithe </s> 
probability of:  5.297608962291984e-32



<s> ged b' fheàrr na Gaidhealtachd . 'S dòcha crìoch a chur gu Gorem grèim ... teine </s> 
probability of:  6.162428820581238e-27



<s> Ach nuair a leigeil ma_sgaoil . ” orra cuir iad ceart uidhir a chaidh air dà innse dha de lemon sole </s> 
probability of:  1.0384685774243857e-33



<s> dè thachair san amharc </s> 
probability of:  7.69054426851154e-08



<s> agus

In [14]:
trigram_model = train_ngram(trigrams)

In [15]:
trigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(trigram_model, ["<s>","<s>"])

    trigram_sentences.append((test,probability))

print_sentences(trigram_sentences)

<s> <s> [5] an robh buntanas ris an traidisean agus comhdheilbh sòichealach àraidh . </s> 
probability of:  4.536765073093456e-09



<s> <s> Cha b’ e Pòsadh Fiollagain bu choireach </s> 
probability of:  4.369120801821048e-06



<s> <s> [1] 's tha an tràchdas </s> 
probability of:  2.100538847029351e-08



<s> <s> chan fheumadh uill mòr gun teagamh as fhearr a chòrdadh ri gillean </s> 
probability of:  1.7377185007242808e-07



<s> <s> Dh’ fheuch e airson </s> 
probability of:  1.6384203006828934e-06



<s> <s> is thuig iad </s> 
probability of:  9.175153683824204e-05



<s> <s> Chaidh aontachadh cuideachd </s> 
probability of:  9.175153683824204e-05



<s> <s> tha ga dhèanamh air . " </s> 
probability of:  8.613290300473664e-09



<s> <s> far 'eil àsan </s> 
probability of:  9.175153683824204e-05



<s> <s> Chaidh Amanda Lynne - aois dà fhichead sa trì - as fhaide air_falbh bhon a’ phàpa </s> 
probability of:  2.3526035086728732e-08





# Smoothed Models

In [16]:
#TODO: replace all the low-frequent words in the training data with the token <UNK>.
# the rest will be similar to the part right preceeding part 1. 
# also report the size of the vocabulary and set the value of lambda to prepare for add-lambda smoothing. 
lambda_value = 0.00001

def replace_low_freq(sentence):
    low_freq = 1

    words_UNK = [word if unigram_fdist[word] > low_freq else "<UNK>" for word in sentence]
  
    return " ".join(words_UNK)

sentences_UNK = [replace_low_freq(sent) for sent in sentences]

for sent in sentences_UNK[0:10]:
    print(sent, end="\n")


def train_ngram_smoothed(ngrams, vocab):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    # vocab = len(set(ngrams)) # make vocab number of N-1 grams

    for ng in ngrams:
        model[ng[:-1]][ng[-1]] += 1  # [(w1, w2, ... , wn-1)][wn]
    
    total_count = 0      
    for hist in model: # hist = (w1, w2, ... , wn-1)
        total_count = float(sum(model[hist].values()))
        # if "<UNK>" in hist:
        #     total_count = 0
        for w in model[hist]: # w = wn
            count = model[hist][w]
            # if w == "<UNK>":
            #     count = 0
            model[hist][w] = (count + lambda_value)/(total_count + lambda_value * vocab)
    
    return model

<s> [3] ach bha e neònach </s>
<s> nach do test iad na [?] leis a' vet - </s>
<s> cha robh duine airson an ceannach </s>
<s> nan robh [?] na laoigh annta </s>
<s> [1] cha robh còir aca an robh idir na <UNK> a leigeil cuide riutha co-dhiubh </s>
<s> bha a' chuid a b' fhearr an dà <UNK> air an cumail ann_an <UNK> </s>
<s> cupa math tea eh ? </s>
<s> [2] àidh , bha mi ' fàs gu math <UNK> a's a' chàr </s>
<s> [1] tha an t- astar gu math fada suas a-siud </s>
<s> <UNK> suas 's sìos </s>


In [17]:
#Todo: extract the smoothed unigram model. same as part 1.1 except for the smoothed probability
unigrams_UNK = [ug for sent in sentences_UNK for ug in sent.split()]
unigrams_count_UNK = len(unigrams_UNK)
unigrams_UNK_vocab = len(set(unigrams_UNK))

unigram_fdist_UNK = nltk.FreqDist(unigrams_UNK)

unigram_model_smoothed = {}
for ug, freq in unigram_fdist_UNK.most_common(200): 
    if ug == "<UNK>":
        freq = 0
    
    unigram_model_smoothed[ug] = (freq + lambda_value)/(unigrams_count_UNK + lambda_value * unigrams_UNK_vocab)

In [18]:
#Todo: extract the smoothed bigram model. same as part 1.2 except for the smoothed probability

bigrams_UNK = [bg for sent in sentences_UNK for bg in nltk.bigrams(sent.split())]
bigrams_UNK_vocab = len(set(bigrams_UNK))

bigram_model_smoothed = train_ngram_smoothed(bigrams_UNK,unigrams_UNK_vocab)

In [19]:
#Todo: extract the smoothed trigram model, same as part 1.3 except for the smoothed probability

#re-padding each sentence with <s><s> at the beginning 
#that is, append extra <s> to the beginning of each sentence

trigrams_UNK = [tg for sent in sentences_UNK for tg in nltk.trigrams(["<s>"] + sent.split())]
trigrams_UNK_vocab = len(set(bigrams_UNK))

trigram_model_smoothed = train_ngram_smoothed(trigrams_UNK,bigrams_UNK_vocab)

In [20]:
unigram_sentences = []

#generate the sentences
for index in range(0,num_sentences):
    test, probability = unigram_random_sentence(unigram_model_smoothed)

    unigram_sentences.append((test,probability))
    
#TODO: generate 10 sentences using the smoothed unigram model. Suppose maximum sentence length = 30
    
print_sentences(unigram_sentences)

<s> t- ars’ bu dhen Ach 'na leth ann_an 's chionn sa no cho ò eile iad anns fhalbh chaidh dìreach mun mi thoirt a-staigh leis robh gun deach do sam_bith </s> 
probability of:  2.0239588943808984e-87



<s> ’s deug gun tha sibh is [5] thàinig thuirt [Placename] aige mòr [3] ga ann_am a ann thàinig aig dè thug ise ' siud taobh coimhead sìos [?] bho an </s> 
probability of:  7.643497253493043e-86



<s> tha coimhead chan [4] ma-thà son e chionn feuchainn i " anns biodh ise àite sinn Chaidh ? mi an [5] sibh [3] fhios eile bho ise anns a-staigh - </s> 
probability of:  1.041783391170368e-84



<s> aca nach nas cha bliadhna [3] a' bu ri dhan latha suas a-mach a-staigh rinn taobh ais bh' air do dè feuchainn tu cho ma dhe ag th' nam sibh </s> 
probability of:  6.815349751995302e-87



<s> as a_h-uile t- bhios feuchainn 'ga nach nuair ann_am 's bhith sibh - [?] 'se ò am leis againn dèanamh do cho dol a's Ach àite no ri às uill </s> 
probability of:  1.3753588455292897e-87



<s> [2] an ò ruith 

In [21]:
bigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(bigram_model_smoothed, ["<s>"])

    bigram_sentences.append((test,probability))

#ToDo: generate 10 sentences using the smoothed bigram model. Suppse maximum sentence length = 30

print_sentences(bigram_sentences)

<s> “ A' cur cabhaig iad iad </s> 
probability of:  1.1245381155099387e-11



<s> thàinig [Name] a_h-uile tè an-diugh neo an-dé </s> 
probability of:  4.611843013514056e-15



<s> dè hearing </s> 
probability of:  1.8469850987308336e-05



<s> ach siud às . " Ged nach do dhealaich </s> 
probability of:  1.7364110369337176e-16



<s> Chaidh fàgail </s> 
probability of:  2.1097760398403164e-05



<s> bidh feadhainn na Lappaich , a' fàs gu leth Albainn faodaidh mi smaoineachadh , </s> 
probability of:  5.810969138787631e-25



<s> dè leabaidh . 'S deagh fheum a bharrachd feum beagan dhaoine a thogail . ' glosaidh </s> 
probability of:  1.3983128156157328e-30



<s> bha ise is a’ mhìos </s> 
probability of:  3.068808235320813e-12



<s> [3] cha do choisich iad e na tha </s> 
probability of:  7.545379838508152e-15



<s> chan fheum iad an_seo fadachd air chall an còrr aca co-dhiubh cha mhair an cuid chloinne anns gach sgillinn dhen Ghàidhlig a thigeadh anail a-mach fearann . Is fhada nis </

In [22]:
trigram_sentences = []

#generate the sentences
for i in range(0,num_sentences):
    test, probability = ngram_random_sentence(trigram_model_smoothed, ["<s>","<s>"])

    trigram_sentences.append((test,probability))

#ToDo: generate 10 sentences using the smoothed trigram model. Suppse maximum sentence length = 30
    
print_sentences(trigram_sentences)

<s> <s> agus bha a_h-uile neach </s> 
probability of:  8.950264677106227e-06



<s> <s> gun dh’ iarr Fionn orra a' lorg a leantainn </s> 
probability of:  1.049367583821434e-06



<s> <s> bidh e agaibh tha mi creidsinn le a cheann rithe </s> 
probability of:  1.1354989630120756e-09



<s> <s> gu faic daoine tòrr a chluinntinn a-staigh 'san taigh , </s> 
probability of:  4.3885917772340603e-07



<s> <s> nach bi ro fhada </s> 
probability of:  1.2461436577841353e-05



<s> <s> gur docha </s> 
probability of:  6.80903189518483e-05



<s> <s> air toll </s> 
probability of:  3.88581300247272e-05



<s> <s> is cha luaithe sin </s> 
probability of:  4.863304869401547e-05



<s> <s> 'S cha robh control aig [Name] an gille dhachaidh </s> 
probability of:  7.540248662355923e-10



<s> <s> is bha e cianail </s> 
probability of:  1.6355596309451576e-07





# Most Probable Sentences

In [23]:
def print_sentences_joined(sent_list):
    for sent, prob in sent_list:
        print(sent)
        print("probability of: ", prob)
        print("\n\n")

In [24]:
bigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(bigram_model_smoothed, ["<s>"],n=2)

    bigram_sentences.append((test,probability))


bigram_sentences = [(" ".join(words), prob) for words, prob in bigram_sentences]
bigram_sentences_most_probable = heapq.nlargest(num_sentences, set(bigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(bigram_sentences_most_probable)

<s> a bha e </s>
probability of:  0.00016615711334258358



<s> a tha e </s>
probability of:  0.00014759279009689586



<s> a bha iad </s>
probability of:  0.00012441214043736523



<s> agus tha e </s>
probability of:  0.00010214525599508602



<s> a tha mi </s>
probability of:  9.101402220723725e-05



<s> agus tha mi </s>
probability of:  6.298851448907068e-05



<s> agus bha e </s>
probability of:  4.116628423583211e-05



<s> agus bha iad </s>
probability of:  3.08237512833599e-05



<s> agus 's e </s>
probability of:  3.076998370482919e-05



<s> agus 's a bha e </s>
probability of:  3.3493079992561964e-07





In [25]:
trigram_sentences = []

#generate the sentences
for i in range(0,10000):
    test, probability = ngram_random_sentence(trigram_model_smoothed, ["<s>","<s>"],n=2)

    trigram_sentences.append((test,probability))


trigram_sentences = [(" ".join(words), prob) for words, prob in trigram_sentences]
trigram_sentences_most_probable = heapq.nlargest(num_sentences, set(trigram_sentences), key=operator.itemgetter(1))
    
print_sentences_joined(trigram_sentences_most_probable)

<s> <s> agus tha mi a' smaoineachadh </s>
probability of:  0.0001989161625325974



<s> <s> agus tha mi a' creidsinn </s>
probability of:  0.00012305711304549973



<s> <s> a tha an rèitire ag ràdh </s>
probability of:  7.681229011733656e-05



<s> <s> agus tha mi ' creidsinn </s>
probability of:  5.4829261985709114e-05



<s> <s> a bha iad ag ràdh </s>
probability of:  3.458942907572759e-05



<s> <s> agus tha mi ' smaoineachadh </s>
probability of:  2.1931752312912793e-05



<s> <s> a tha a' dol dha gabhail </s>
probability of:  1.4167332522354539e-05



<s> <s> agus bha iad ag ràdh </s>
probability of:  1.3524907069541053e-05



<s> <s> agus tha mi a' smaoineachadh [?] </s>
probability of:  1.212691073887105e-05



<s> <s> a tha an rèitire ag ràdh ; </s>
probability of:  4.6075393767911364e-06





In [26]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('<s>','Tha')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('e', 0.0819302339746966), ('an', 0.07562791070830581), ('na', 0.05041861764274266), ('seo', 0.04096513274315647)]


In [27]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('Tha', 'an')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('t-', 0.15184170238417447), ('dà', 0.11388137168895761), ('searg', 0.07592104099374072), ('gluasad', 0.03796071029852384)]


In [28]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('an','t-')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('àite', 0.042491162590613904), ('seachdainn', 0.042491162590613904), ('saoghail', 0.042491162590613904), ('ainm', 0.03824205058066452)]


In [29]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('t-','àite')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('</s>', 0.40507919687538735), ('glan', 0.08101648750049623), ('mòr', 0.08101648750049623), ('am', 0.08101648750049623), ('na', 0.08101648750049623)]


In [30]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('àite','mòr')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('ùr', 0.42675469105403085), ('a', 0.42675469105403085)]


## Dè test

In [31]:
most_probable = heapq.nlargest(5, trigram_model_smoothed[('<s>','Dè')].items(), key=operator.itemgetter(1))

most_probable = [w for w in most_probable if w[0] != '<UNK>']
print(most_probable)

[('do', 0.29910955974504155), ('am', 0.29910955974504155), ('fon', 0.29910955974504155)]
